In [17]:
import pandas as pd
import random
import math

df = pd.read_csv('trial.csv')
df.head()

,UID,Course
0,201700023,DL
1,2021700026,DL
2,2020400043,DL
3,2021600038,DL
4,2021600043,DL


In [18]:
# Sample data
students = df['UID'].tolist()
courses = df['Course'].tolist()
rooms = ['609', '601', '701', '605', '603']

In [19]:
# Initial configuration (random assignment)
def random_assignment(students):
    assignment = list(range(len(students)))  # Seats are 0, 1, 2, ...
    random.shuffle(assignment)  # Shuffle to create random assignment
    return assignment

# Objective function to calculate the number of adjacent same-course pairs
def objective_function(assignment, courses):
    conflicts = 0
    for i in range(len(assignment) - 1):
        if courses[assignment[i]] == courses[assignment[i + 1]]:
            conflicts += 1
    return conflicts

# Generate a neighbor solution by swapping two students
def generate_neighbor(assignment):
    neighbor = assignment[:]
    idx1, idx2 = random.sample(range(len(assignment)), 2)
    neighbor[idx1], neighbor[idx2] = neighbor[idx2], neighbor[idx1]
    return neighbor

# Simulated annealing algorithm
def simulated_annealing(students, courses, initial_temp=1000, cooling_rate=0.99, max_iterations=10000):
    current_solution = random_assignment(students)
    current_cost = objective_function(current_solution, courses)
    best_solution = current_solution[:]
    best_cost = current_cost

    temperature = initial_temp

    for iteration in range(max_iterations):
        neighbor = generate_neighbor(current_solution)
        neighbor_cost = objective_function(neighbor, courses)

        # Calculate the cost difference
        cost_diff = neighbor_cost - current_cost

        # Decide whether to accept the neighbor solution
        if cost_diff < 0 or random.random() < math.exp(-cost_diff / temperature):
            current_solution = neighbor
            current_cost = neighbor_cost

            # Update the best solution found
            if current_cost < best_cost:
                best_solution = current_solution[:]
                best_cost = current_cost

        # Cool down the temperature
        temperature *= cooling_rate

    return best_solution, best_cost

# Run the simulated annealing algorithm
best_assignment, best_conflicts = simulated_annealing(students, courses)

# Output the result
# print("Best Seat Assignment:")
# for seat, student_idx in enumerate(best_assignment):
#     print(f"Seat {seat + 1}: {students[student_idx]} (Course: {courses[student_idx]})")
# print(f"Total adjacent conflicts: {best_conflicts}")

In [27]:
import csv

FILE = open('seat.csv', 'w')
writer = csv.writer(FILE)

In [28]:
current_capacity = 0
current_room = 0
limit = 35
writer.writerow(['Seat', 'UID', 'Course', 'Room'])

for seat, student_idx in enumerate(best_assignment):
    if current_capacity>limit:
        current_room+=1
        current_capacity=0
    current_capacity+=1
    writer.writerow([((seat)%limit)+1, students[student_idx], courses[student_idx], rooms[current_room]])

FILE.close()